# Getting started with Cirq and Quantinuum on Azure Quantum

This notebook shows how to send a basic quantum circuit expressed using the [Cirq library](https://quantumai.google/cirq) to a Quantinuum target via the Azure Quantum service.

First, install `azure-quantum` with the Cirq dependencies.

In [1]:
# To install the Azure Quantum client package with support for
# Cirq, uncomment and run the line below:
#
# !pip install azure-quantum[cirq]==0.19.2109.165653 --quiet

## Connecting to the Azure Quantum service

To connect to the Azure Quantum service, find the resource ID and location of your Workspace from the Azure portal here: https://portal.azure.com. Navigate to your Azure Quantum workspace and copy the values from the header.

<img src="../azure-quantum-resource-id.png" alt="How to retrieve the resource ID and location from an Azure Quantum workspace">

In [2]:
from azure.quantum.cirq import AzureQuantumService
service = AzureQuantumService(
    resource_id="",
    location="",
    default_target="quantinuum.sim.h1-1sc"
)

### List all targets

You can now list all the targets that you have access to, including the current queue time and availability. Running this method will trigger authentication to your Microsoft account, if you're not already logged in.

In [3]:
service.targets()

[<Target name="ionq.qpu", avg. queue time=250 s, Available>,
 <Target name="ionq.simulator", avg. queue time=0 s, Available>,
 <Target name="quantinuum.qpu.h1-1", avg. queue time=0 s, Unavailable>,
 <Target name="quantinuum.sim.h1-1sc", avg. queue time=0 s, Available>,
 <Target name="quantinuum.qpu.h1-2", avg. queue time=93509 s, Available>,
 <Target name="quantinuum.sim.h1-2sc", avg. queue time=0 s, Available>,
 <Target name="quantinuum.sim.h1-1e", avg. queue time=26 s, Available>]

## Run a simple circuit

Now, let's create a simple Cirq circuit to run.

This notebook assumes some familiarity with Cirq. To read more about Cirq, review the [Cirq documentation](https://quantumai.google/cirq).

In [4]:
import cirq

q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit(
    cirq.H(q0), # Hadamard
    cirq.CNOT(q0, q1), # CNOT
    cirq.measure(q0, q1, key='b') # Measure both qubits
)
circuit

0: ───H───@───M('b')───
          │   │
1: ───────X───M────────

You can now run the program via the Azure Quantum service and get the result. The following cell will submit a job that runs the circuit with 100 shots, wait until the job is completed and return the results.

In [5]:
result = service.run(program=circuit, repetitions=100)

.......

This returns a `cirq.Result` object. Note that we used the API validator provided by Quantinuum, which only returns zeros.

In [6]:
print(result)

b=0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


## Asynchronous workflow using Jobs

For long-running circuits, it can be useful to run them asynchronously. The `service.create_job` method returns a `Job`, which you can use to get the results after the job has run successfully.

In [7]:
job = service.create_job(
    program=circuit,
    repetitions=100
)

To check on the job status, use `job.status()`:

In [8]:
job.status()

'Waiting'

To wait for the job to be done and get the results, use the blocking call `job.results()`:

In [9]:
result = job.results()
print(result)

.......{'m_b': ['00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00']}


Note that this does not return a `cirq.Result` object. Instead it returns a dictionary that is specific to the Quantinuum simulator.

In [10]:
type(result)

dict

In [11]:
# Print package versions used in this notebook
import cirq
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

cirq==0.12.0
